# Simpeg 2D inversion Using MTpy-v2 tools

This is an example of how to use MTpy-v2 tools to integrate with Simpeg to build an inversion.  

The recipe in MTpy-v2 is to create a quadree mesh from the given data, then invert the data using computed data errors




In [ ]:
from mtpy_data import FWD_NE_FAULTS_PROFILE_LIST

from mtpy import MTData
from mtpy.modeling.simpeg.recipes import Simpeg2D